In [1]:
from src.util import data_from_many
import os
from json import loads
import re
import datetime

publication = 'wsj'

datafiles = ['/datapool/news_articles/raw_data/'+publication+'/' + fname 
             for fname in os.listdir('/datapool/news_articles/raw_data/'+publication+'/') if '.json' in fname]

data = map(loads,data_from_many(datafiles))

In [2]:
wsj_date = re.compile('(?<=content=")\d{4}-\d{2}-\d{2}')
def get_wsj_date(html):
    try:
        match = wsj_date.search(html)
        return datetime.datetime.strptime(match.group(), '%Y-%m-%d').date()
    except:
        return None
    
from collections import Counter
wsj_dates = Counter()

i = 0
for item in data:
    i+=1
    date = get_wsj_date(item['html'])
    if date:
        wsj_dates.update([date.isoformat()])
    if i % 1000 == 0:
        print(i,end = '\r')

In [3]:
with open(publication+'_datelist.csv','w') as file:
    for item in sorted(list(wsj_dates.items()),key = lambda x: -int(x[0].split('-')[0])):
        file.write('%s, %i\n' % item)